In [1]:
#pip install datasets

In [2]:

import argparse
from modules.preprocess import preprocessing

from modules.vocab import KoreanSpeechVocabulary
from modules.data import load_dataset

import pandas as pd

c:\Users\kdy\anaconda3\envs\kwav2vec\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kdy\anaconda3\envs\kwav2vec\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
args = argparse.ArgumentParser()
args.add_argument('--dataset', default='Video', type=str)
args.add_argument('--train_dir', default='train', type=str)
args.add_argument('--batch_size', default=128, type=int)
args.add_argument('--lr', default=0.001, type=float)
args.add_argument('--maxlen', default=50, type=int)
args.add_argument('--hidden_units', default=50, type=int)
args.add_argument('--num_blocks', default=2, type=int)
args.add_argument('--num_epochs', default=201, type=int)
args.add_argument('--num_heads', default=1, type=int)
args.add_argument('--dropout_rate', default=0.5, type=float)


args.add_argument('--audio_extension', type=str, default='pcm')
args.add_argument('--transform_method', type=str, default='fbank')
args.add_argument('--feature_extract_by', type=str, default='kaldi')
args.add_argument('--sample_rate', type=int, default=16000)
args.add_argument('--frame_length', type=int, default=20)
args.add_argument('--frame_shift', type=int, default=10)
args.add_argument('--n_mels', type=int, default=80)
args.add_argument('--freq_mask_para', type=int, default=18)
args.add_argument('--time_mask_num', type=int, default=4)
args.add_argument('--freq_mask_num', type=int, default=2)
args.add_argument('--normalize', type=bool, default=True)
args.add_argument('--del_silence', type=bool, default=True)
args.add_argument('--spec_augment', type=bool, default=True)
args.add_argument('--input_reverse', type=bool, default=False)


config = args.parse_args(args=[])

In [4]:
DATASET_PATH="/Users/kdy/2022-Korean-AI-Competition/data/t2-conf"

In [5]:
config.dataset_path = os.path.join(DATASET_PATH, 'train', 'train_data')
label_path = os.path.join(DATASET_PATH, 'train', 'train_label')
preprocessing(label_path, os.getcwd())
dataset = load_dataset(os.path.join(os.getcwd(), 'transcripts.txt'))

[INFO] create_script started..
[INFO] Preprocessing is Done


In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(train_dataset.remove_columns(["path"]), num_examples=20)

,text
0,신호가 적은 길로 알려 줘.
1,어린이용 오디오북 콘텐츠 검색해 줘.
2,지금 날씨는 좀 나아졌는지 알아 봐 줘.
3,주말에 반팔 입어도 되는 날씨인지 궁금해.
4,우리집 조명 다 꺼.
5,지금 차가 막히는데 우회도로 알려 줘.
6,에너지 절약 모드 실행 부탁해.
7,밤이니까 불 점등해 줘.
8,tv 전원 연결해 줘.
9,경기 일정 요약해 줘.


In [9]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\…]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [10]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

100%|██████████| 200/200 [00:00<00:00, 10001.92ex/s]


In [11]:
show_random_elements(train_dataset.remove_columns(["path"]))

,text
0,오늘 외환 시세 조합정보 요청합니다
1,경부 고속 도로에 사고 소식 확인해 줄래
2,등 켜 줘
3,교통량을 확인해야 해
4,이번 주말 축구 경기 몇시에 있어요
5,환율 따져 줘
6,씻기 전에 커피 한 잔 마시고 싶어
7,날씨 어떤지 봐줄 수 있어
8,이제 에너지 절약 모드로 바꿔
9,출근길 빨리 가는 경로 말해 주세요


---

In [12]:
def extract_all_chars(batch):
  #print(batch)
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [13]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

100%|██████████| 1/1 [00:00<00:00, 502.31ba/s]


In [14]:
vocab_list = sorted(set(vocab_train["vocab"][0])| set(vocab_test["vocab"][0]))

In [15]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [17]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

563

In [18]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

---

In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [21]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

---

In [33]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [34]:
train_dataset = train_dataset.map(speech_file_to_array_fn, remove_columns=train_dataset.column_names)

100%|██████████| 799/799 [00:00<00:00, 850.01ex/s]


In [35]:
test_dataset = test_dataset.map(speech_file_to_array_fn, remove_columns=test_dataset.column_names)

100%|██████████| 200/200 [00:00<00:00, 727.26ex/s]


In [36]:
train_dataset["sampling_rate"][:10]

[48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000]

In [37]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [38]:
train_dataset = train_dataset.map(resample)
test_dataset = test_dataset.map(resample)

  0%|          | 0/799 [00:00<?, ?ex/s]C:\Users\kdy\AppData\Local\Temp\ipykernel_37344\1467501469.py:5: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
  0%|          | 1/799 [00:01<15:54,  1.20s/ex]C:\Users\kdy\AppData\Local\Temp\ipykernel_37344\1467501469.py:5: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
100%|██████████| 200/200 [00:31<00:00,  6.31ex/s]


In [ ]:
rand_int = random.randint(0, len(train_dataset))
train_dataset[rand_int]["speech"]

In [46]:
print("Target text:", train_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", train_dataset[rand_int]["sampling_rate"])

Target text: 지금 텔레비전 좀 켜 줘 
Input array shape: (56640,)
Sampling rate: 16000


---

In [48]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [49]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, batch_size=8, num_proc=1, batched=True)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, batch_size=8, num_proc=1, batched=True)

  0%|          | 0/100 [00:00<?, ?ba/s]c:\Users\kdy\anaconda3\envs\kwav2vec\lib\site-packages\transformers\feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
100%|██████████| 25/25 [00:07<00:00,  3.53ba/s]


In [50]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [51]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [52]:
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'quantizer.codevectors', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir="container_0/ckpts/",
  logging_dir = "container_0/runs/",
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=4e-4,
  warmup_steps=int(0.1*1320), #10%
  save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("container_0/wav2vec2-large-xlsr-kn")

---

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# test_dataset = #TODO: WRITE YOUR CODE TO LOAD THE TEST DATASET. For sample see the Colab link in Training Section.

processor = Wav2Vec2Processor.from_pretrained("container_0/wav2vec2-large-xlsr-kn")
model = Wav2Vec2ForCTC.from_pretrained("container_0/wav2vec2-large-xlsr-kn")

resampler = torchaudio.transforms.Resample(48_000, 16_000) # The original data was with 48,000 sampling rate. You can change it according to your input.

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["text"][:2])